# MNIST Object detection using YOLOv4

In [9]:
import os

In [10]:
!git submodule update --init --recursive
%pip install -r MNIST-ObjectDetection/requirements.txt yolov4
if not os.path.exists('MNIST-ObjectDetection/data'):
    %cd MNIST-ObjectDetection
    !python generate_data.py
    %cd ..

Note: you may need to restart the kernel to use updated packages.


In [11]:
GPU_NAME =  str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read()).strip()
GPU_COMPUTE_CAPABILITY = str(os.popen('nvidia-smi --query-gpu=compute_cap --format=csv,noheader').read()).strip()
# Remove newlines
GPU_COMPUTE_CAPABILITY = GPU_COMPUTE_CAPABILITY.replace('.', '')

print(f'GPU_NAME: {GPU_NAME}')
print(f'GPU_COMPUTE_CAPABILITY: {GPU_COMPUTE_CAPABILITY}')
# Build using make, enable OpenCV and CUDNN
import re
with open('darknet/Makefile', 'r') as f:
    makefile = f.read()
makefile = re.sub(r'GPU=0', 'GPU=1', makefile)
makefile = re.sub(r'CUDNN=0', 'CUDNN=1', makefile)
makefile = re.sub(r'CUDNN_HALF=0', 'CUDNN_HALF=1', makefile)
makefile = re.sub(r'OPENCV=0', 'OPENCV=1', makefile)

# Remove everything in arch= and remove lines after it if it ends with a \. Replace with GPU_COMPUTE_CAPABILITY
makefile = re.sub(r'ARCH=(.*\\\n)*.*', f'ARCH={GPU_COMPUTE_CAPABILITY}', makefile)


# Enable CUDNN_HALF if GPU compute capability >= 75
if int(GPU_COMPUTE_CAPABILITY) >= 75:
    makefile = re.sub(r'CUDNN_HALF=0', 'CUDNN_HALF=1', makefile)

with open('darknet/Makefile', 'w') as f:
    f.write(makefile)

# Build darknet
!(cd darknet && make -j)

GPU_NAME: NVIDIA GeForce RTX 4070 Ti
GPU_COMPUTE_CAPABILITY: 89
mkdir -p ./obj/
chmod +x *.sh


process_begin: CreateProcess(NULL, uname, ...) failed.
Makefile:22: pipe: No error
La syntaxe de la commande n'est pas correcte.
process_begin: CreateProcess(NULL, chmod +x *.sh, ...) failed.
make (e=2): Le fichier sp�cifi� est introuvable.

make: *** [Makefile:191: setchmod] Error 2
make: *** Waiting for unfinished jobs....
make: *** [Makefile:185: obj/] Error 1


In [1]:
%cd darknet


c:\Users\aurel\ING3-DNN-Projet\darknet


In [12]:
# Upload video to colab
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

ModuleNotFoundError: No module named 'google.colab'

In [8]:
if not os.path.exists("yolov4.weights"):
  !curl https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4.weights -o yolov4.weights

# Load darknet dll
import ctypes
import os
import sys
import cv2
import darknet.darknet as darknet

# Print darknet path
print(darknet.__version__)

# Load network
net = darknet.load_net(b"darknet/cfg/yolov4.cfg", b"yolov4.weights", 0)
meta = darknet.load_meta(b"darknet/cfg/coco.data")
 
# Run inference and save output to output.mp4 using opencv
darknet_image = darknet.make_image(darknet.network_width(net), darknet.network_height(net),3)
class_colors = darknet.load_colors(b"darknet/data/colors.txt")
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb,
                               (darknet.network_width(net), darknet.network_height(net)),
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image, frame_resized.tobytes())
    detections = darknet.detect_image(net, meta, darknet_image, thresh=0.25)
    image = darknet.draw_boxes(detections, frame_resized, class_colors)

    out.write(image)

cap.release()
out.release()



c:\Users\aurel\ING3-DNN-Projet\darknet


FileNotFoundError: Could not find module 'c:\Users\aurel\ING3-DNN-Projet\darknet\darknet.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
# Use YOLOv4 to detect MNIST digits from images

# Structure is train -> images -> 0,1,2,3,4,5,6,7,8,9 -> 0.jpg, 1.jpg, 2.jpg, ...
# Structure is train -> labels -> 0,1,2,3,4,5,6,7,8,9 -> 0.txt, 1.txt, 2.txt, ...
# Same for test

data_path = 'MNIST-ObjectDetection/data/mnist_detection'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

# Train YOLOv4 on MNIST dataset